## Black-Scholes Formulas
### d1 and d2
#### $ d_1 = \frac{ln(\frac{S}{K}) + (r - q + \frac{\sigma^2}{2})*T}{\sigma\sqrt{T}} $
#### $ d_2 = d_1 - \sigma\sqrt{T} $

### Option Price
#### For Call Options: $ C = Se^{-qT}N(d_1) - Ke^{-rT}N(d_2) $ 
#### For Put Options: $ P = Ke^{-rT}N(-d_2) - Se^{-qT}N(-d_1) $

## Greeks
### Delta($\Delta = \frac{\delta V}{\delta S}$): 
#### For Call Options: $ \Delta_c = e^{-qT}N(d_1) $ 
#### For Put Options: $ \Delta_p = -e^{-qT}N(-d_1) $

### Gamma($\Gamma = \frac{\delta^2 V}{\delta S^2}$):
#### $ \Gamma = \frac{e^{-qT}N'(d_1)}{S\sigma\sqrt{T}} $

### Vega($\nu = \frac{\delta V}{\delta \sigma}$):
#### $ \nu = 0.01 * Se^{-qT}N'(d_1)\sqrt{T} $

### Theta($\Theta = \frac{\delta V}{\delta T}$):
#### For Call Options: $ \theta_c = \frac{1}{365} * \left(-\frac{Se^{-qT}N'(d_1)\sigma}{2\sqrt{T}} + qSe^{-qT}N(d_1) - rKe^{-rT}N(d_2)\right) $
#### For Put Options: $ \theta_p = \frac{1}{252} * \left(-\frac{Se^{-qT}N'(d_1)\sigma}{2\sqrt{T}} - qSe^{-qT}N(-d_1) + rKe^{-rT}N(-d_2)\right) $

### Rho($\rho = \frac{\delta V}{\delta r}$):
#### For Call Options: $ \rho_c = 0.01 * KTe^{-rT}N(d_2) $
#### For Put Options: $ \rho_p = -0.01 * KTe^{-rT}N(-d_2) $

#### N(d) is the cumulative distribution function (cdf) of standard normal distribution,
#### N'(d) is the probability density function (pdf) of standard normal distribution,
#### Theta is expressed as the daily rate of time decay, so we divided by 252 to account for number of trading days per year.

In [199]:
import numpy as np
from scipy.stats import norm
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Functions

In [317]:
def BlackScholes(r, S, K, T, sigma, option_type="C", q=0):
    """
Calculate the price of a European Call or Put option using the Black-Scholes formula, including adjustments for continuous dividend yield.

Parameters
----------
r : float
    Risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
S : float
    Current stock price.
K : float
    Strike price of the option.
T : float
    Time to expiration of the option, expressed in years (e.g., 0.5 for six months).
sigma : float
    Volatility of the underlying stock, expressed as a decimal.
option_type : str, optional
    Type of the option: 'C' for Call option or 'P' for Put option. The default is 'C'.
q : float, optional
    Continuous dividend yield of the underlying stock, expressed as a decimal. The default is 0.

Returns
-------
None
    This function prints the option price, Delta, Gamma, Vega, Theta, and Rho for the specified Call or Put option.
    - Option Price: The theoretical price of the option using the Black-Scholes formula.
    - Delta: Measures the rate of change of the option's price with respect to changes in the underlying asset's price.
    - Gamma: Measures the rate of change in Delta with respect to changes in the underlying asset's price.
    - Vega: Measures sensitivity of the option's price to changes in the volatility of the underlying asset. Note that Vega is not a Greek letter; the symbol ν (nu) is sometimes used.
    - Theta: Measures the rate of time decay of the option's price, expressed per day. Used 252 trading days.
    - Rho: Measures sensitivity of the option's price to changes in the risk-free interest rate.

Note
----
This function assumes European options, which can only be exercised at expiration. It does not apply to American options, which can be exercised at any time before expiration.
----
"""
    
    # Define d1 and d2 for Black-Scholes, including the dividend yield
    d1 = (np.log(S / K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Subfunction to calculate delta
    def calc_delta():
        if option_type == "C":
            return norm.cdf(d1, 0, 1)  # Delta for Call option
        elif option_type == "P":
            return -norm.cdf(-d1, 0, 1)  # Delta for Put option
        
    # Subfunction to calculate Gamma
    def calc_gamma():
        return norm.pdf(d1, 0, 1) / (S * sigma * np.sqrt(T))
    
    # Subfunction to calculate Vega
    def calc_vega():
        return (S * norm.pdf(d1, 0, 1) * np.sqrt(T)) * 0.01
    
    # Subfunction to calculate Theta, including dividend yield
    def calc_theta():
        if option_type == "C":
            return (-S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) + q * S * norm.cdf(d1, 0, 1) - r * K * np.exp(-r * T) * norm.cdf(d2, 0, 1)) / 252 # Theta for Call option
        elif option_type == "P":
            return (-S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) - q * S * norm.cdf(-d1, 0, 1) + r * K * np.exp(-r * T) * norm.cdf(-d2, 0, 1)) / 252 # Theta for Put option

    # Subfunction to calculate Rho
    def calc_rho():
        if option_type == "C":
            return (K * T * np.exp(-r * T) * norm.cdf(d2, 0, 1)) * 0.01  # Rho for Call option
        elif option_type == "P":
            return (-K * T * np.exp(-r * T) * norm.cdf(-d2, 0, 1)) * 0.01  # Rho for Put option

    try:
        delta = calc_delta()  # Call the subfunction to calculate delta
        gamma = calc_gamma()  # Call the subfunction to calculate gamma
        vega = calc_vega()    # Call the subfunction to calculate vega
        theta = calc_theta()  # Call the subfunction to calculate theta
        rho = calc_rho()      # Call the subfunction to calculate rho
        if option_type == "C":
            price = S * np.exp(-q * T) * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1)
            print(f"Call Option Price: {price:.5f}")
            print("-----------------")
            print(f"Call Delta: {delta:.5f}")
            print(f"Gamma: {gamma:.5f}")
            print(f"Vega: {vega:.5f}")
            print(f"Call Theta: {theta:.5f}")
            print(f"Call Rho: {rho:.5f}")
        elif option_type == "P":
            price = K * np.exp(-r * T) * norm.cdf(-d2, 0, 1) - S * np.exp(-q * T) * norm.cdf(-d1, 0, 1)
            print(f"Put Option Price: {price:.5f}")
            print("-----------------")
            print(f"Put Delta: {delta:.5f}")
            print(f"Gamma: {gamma:.5f}")
            print(f"Vega: {vega:.5f}")
            print(f"Put Theta: {theta:.5f}")
            print(f"Put Rho: {rho:.5f}")
        else:
            raise ValueError("Invalid option type! Please confirm option type either 'C' for Call option or 'P' for Put option")
    except ValueError as e:
        print(e)
        
def delta_hedge(stock_price, num_shares, option_price, option_delta, strike_price, new_stock_price, option_type="P"):
    """
    Calculate the gain/loss from the stock, the cost of the hedge, and the gain/loss from the options.

    Parameters
    ----------
    stock_price : float
        Current price of the stock.
    num_shares : int
        Number of shares held.
    option_price : float
        Price of the option.
    option_delta : float
        Delta of the option.
    strike_price : float
        Strike price of the option.
    new_stock_price : float
        New price of the stock after the change.
    option_type : str, optional
        Type of the option: 'C' for Call option or 'P' for Put option. The default is 'P'.

    Returns
    -------
    None
        This function prints the gain/loss from the stock, the cost of the hedge, and the gain/loss from the options.
    """
    
    # Calculate the number of options needed to hedge
    num_options = num_shares / abs(option_delta)
    
    # Calculate the cost of the hedge
    cost_of_hedge = num_options * option_price
    
    # Calculate the gain/loss from the stock
    stock_gain_loss = (new_stock_price - stock_price) * num_shares
    
    # Calculate the gain/loss from the options
    if option_type == "C":
        option_gain_loss = max(0, new_stock_price - strike_price) * num_options
    elif option_type == "P":
        option_gain_loss = max(0, strike_price - new_stock_price) * num_options
    else:
        raise ValueError("Invalid option type! Please confirm option type either 'C' for Call option or 'P' for Put option")
    
    # Calculate the net position
    net_position = option_gain_loss + stock_gain_loss - cost_of_hedge
    
    # Print the results
    print(f"Number of Options to Buy: {num_options:.2f}")
    print(f"Stock Gain/Loss: ${stock_gain_loss:.2f}")
    print(f"Cost of Hedge: ${cost_of_hedge:.2f}")
    print(f"Option Gain/Loss: ${option_gain_loss:.2f}")
    print(f"Net Position: ${net_position:.2f}")
    
def calculate_monthly_payment(principal, annual_interest_rate, loan_term_years):
    """
    Calculate the monthly payment amount for a loan.

    Parameters
    ----------
    principal : float
        The amount you want to borrow.
    annual_interest_rate : float
        The annual interest rate as a decimal (e.g., 0.05 for 5%).
    loan_term_years : int
        The loan term in years.

    Returns
    -------
    float
        The monthly payment amount.
    """
    # Convert annual interest rate to monthly interest rate
    monthly_interest_rate = annual_interest_rate / 12
    
    # Calculate the number of months
    num_months = loan_term_years * 12
    
    # Calculate the monthly payment amount using the loan amortization formula
    if monthly_interest_rate > 0:
        monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_months)
    else:
        monthly_payment = principal / num_months
    
    print(f"Monthly Payment: ${monthly_payment:.2f}")
    
def straddle_strategy(r, S, K, T, sigma, call_price, put_price, new_stock_price, num_options=1, q=0):
    """
    Implement a straddle strategy by calculating the price, Delta, Gamma, Vega, Theta, and Rho for both Call and Put options.
    Also, calculate the expected return minus the cost of options.

    Parameters
    ----------
    r : float
        Risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
    S : float
        Current stock price.
    K : float
        Strike price of the options.
    T : float
        Time to expiration of the options, expressed in years (e.g., 0.5 for six months).
    sigma : float
        Volatility of the underlying stock, expressed as a decimal.
    call_price : float
        Price of the Call option.
    put_price : float
        Price of the Put option.
    new_stock_price : float
        New stock price after some period.
    num_options : int, optional
        Number of options in the straddle strategy. The default is 1.
    q : float, optional
        Continuous dividend yield of the underlying stock, expressed as a decimal. The default is 0.

    Returns
    -------
    None
        This function prints the combined Call and Put option prices, Delta, Gamma, Vega, Theta, and Rho.
        It also prints the expected return minus the cost of options.
    """

    # Call the Black-Scholes function for Call option
    BlackScholes(r, S, K, T, sigma, option_type="C", q=q)

    # Call the Black-Scholes function for Put option
    BlackScholes(r, S, K, T, sigma, option_type="P", q=q)

    # Calculate the payoff from the call and put options
    call_payoff = max(0, new_stock_price - K) * num_options
    put_payoff = max(0, K - new_stock_price) * num_options

    # Calculate the total payoff from the straddle strategy
    total_payoff = call_payoff + put_payoff

    # Calculate the total cost of the straddle strategy
    total_cost = (call_price + put_price) * num_options

    # Calculate the expected return minus the cost of options
    expected_return = total_payoff - total_cost

    # Print the results
    print(f"New Stock Price: {new_stock_price}")
    print(f"Call Option Payoff: ${call_payoff:.2f}")
    print(f"Put Option Payoff: ${put_payoff:.2f}")
    print(f"------------------")
    print(f"Total Payoff: ${total_payoff:.2f}")
    print(f"Total Cost of Options: ${total_cost:.2f}")
    print(f"Expected Return Minus Cost of Options: ${expected_return:.2f}")
    
def strangle_strategy(r, S, K_call, K_put, T, sigma, call_price, put_price, new_stock_price, num_options=1, q=0):
    """
    Implement a strangle strategy by calculating the price, Delta, Gamma, Vega, Theta, and Rho for both Call and Put options.
    Also, calculate the expected return minus the cost of options.

    Parameters
    ----------
    r : float
        Risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
    S : float
        Current stock price.
    K_call : float
        Strike price of the Call option.
    K_put : float
        Strike price of the Put option.
    T : float
        Time to expiration of the options, expressed in years (e.g., 0.5 for six months).
    sigma : float
        Volatility of the underlying stock, expressed as a decimal.
    call_price : float
        Price of the Call option.
    put_price : float
        Price of the Put option.
    new_stock_price : float
        New stock price after some period.
    num_options : int, optional
        Number of options in the strangle strategy. The default is 1.
    q : float, optional
        Continuous dividend yield of the underlying stock, expressed as a decimal. The default is 0.

    Returns
    -------
    None
        This function prints the combined Call and Put option prices, Delta, Gamma, Vega, Theta, and Rho.
        It also prints the expected return minus the cost of options.
    """

    # Call the Black-Scholes function for Call option
    BlackScholes(r, S, K_call, T, sigma, option_type="C", q=q)

    # Call the Black-Scholes function for Put option
    BlackScholes(r, S, K_put, T, sigma, option_type="P", q=q)

    # Calculate the payoff from the call and put options
    call_payoff = max(0, new_stock_price - K_call) * num_options
    put_payoff = max(0, K_put - new_stock_price) * num_options

    # Calculate the total payoff from the strangle strategy
    total_payoff = call_payoff + put_payoff

    # Calculate the total cost of the strangle strategy
    total_cost = (call_price + put_price) * num_options

    # Calculate the expected return minus the cost of options
    expected_return = total_payoff - total_cost

    # Print the results
    print(f"New Stock Price: {new_stock_price}")
    print(f"Call Option Payoff: ${call_payoff:.2f}")
    print(f"Put Option Payoff: ${put_payoff:.2f}")
    print(f"------------------")
    print(f"Total Payoff: ${total_payoff:.2f}")
    print(f"Total Cost of Options: ${total_cost:.2f}")
    print(f"Expected Return Minus Cost of Options: ${expected_return:.2f}")

# Calculations

In [354]:
stock_price = 30.87
sigma = 0.0313
rf = yf.Ticker("^TNX").fast_info['last_price'] / 100

In [358]:
BlackScholes(rf, stock_price, 30, 0.08, sigma, "P")

Put Option Price: 0.00001
-----------------
Put Delta: -0.00015
Gamma: 0.00208
Vega: 0.00005
Put Theta: -0.00000
Put Rho: -0.00000


In [359]:
delta_hedge(stock_price=stock_price, num_shares=1000, option_price=1.74, option_delta=-0.5, strike_price=30, new_stock_price=37, option_type="P")

Number of Options to Buy: 2000.00
Stock Gain/Loss: $6130.00
Cost of Hedge: $3480.00
Option Gain/Loss: $0.00
Net Position: $2650.00


In [347]:
calculate_monthly_payment(33612, 0.055, 1)

Monthly Payment: $2885.15


In [348]:
3387.99 - 2885.15

502.8399999999997

In [353]:
straddle_strategy(rf, stock_price, 32, 0.08, sigma, 0.87, 0.72, 37, 100)

Call Option Price: 0.00001
-----------------
Call Delta: 0.00012
Gamma: 0.00174
Vega: 0.00004
Call Theta: -0.00000
Call Rho: 0.00000
Put Option Price: 1.02065
-----------------
Put Delta: -0.99988
Gamma: 0.00174
Vega: 0.00004
Put Theta: 0.00541
Put Rho: -0.02551
New Stock Price: 37
Call Option Payoff: $500.00
Put Option Payoff: $0.00
------------------
Total Payoff: $500.00
Total Cost of Options: $159.00
Expected Return Minus Cost of Options: $341.00


In [362]:
strangle_strategy(rf, stock_price, 32, 30, 0.08, sigma, 0.87, 0.72, 37, 1000)

Call Option Price: 0.00001
-----------------
Call Delta: 0.00012
Gamma: 0.00174
Vega: 0.00004
Call Theta: -0.00000
Call Rho: 0.00000
Put Option Price: 0.00001
-----------------
Put Delta: -0.00015
Gamma: 0.00208
Vega: 0.00005
Put Theta: -0.00000
Put Rho: -0.00000
New Stock Price: 37
Call Option Payoff: $5000.00
Put Option Payoff: $0.00
------------------
Total Payoff: $5000.00
Total Cost of Options: $1590.00
Expected Return Minus Cost of Options: $3410.00
